# Coffee Object Detection Demo

<!-- <img src="https://pages.databricks.com/rs/094-YMS-629/images/SBUX-DB.png"/> -->

This notebook provides a simplified example of object detection (e.g. coffee, cups, chair, etc.) within images and videos.  In addition to use *Databricks Runtime for ML - GPU*, we are also working with the [Image AI](https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl) library.

## Setup
There are additional installations to include the **pre-trained** models as part of **[ImageAI](https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/VIDEO.md):**
* **Yolo V3**: You only look once (YOLO) is a state-of-the-art, real-time object detection system; we are using V3 as noted in [YOLOv3: An Incremental Improvement](https://pjreddie.com/media/files/papers/YOLOv3.pdf) (using this model for images)
* **Resnet**: Residual Network model (in h5 format) that we are using for feature extraction (using this model for videos)

In [0]:
%sh
## Install the ImageAI pre-trained models: YoloV3, ResNet
# wget -P /dbfs/tmp/dennylee/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5
# wget -P /dbfs/tmp/dennylee/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5

In [0]:
## ------------------------------
## Miscellaneous Functions
## ------------------------------
# displayVid(): Shows video from mounted cloud storage
def displayVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/files/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayDbfsVid(): Shows video from DBFS
def displayDbfsVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/dbfs/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayImg(): Shows image from dbfs/cloud storage
def displayImg(filepath):
  dbutils.fs.cp(filepath, "FileStore/%s" % filepath)
  return displayHTML("""
  <img src="/files/%s" width="800">
  """ % filepath)

In [0]:
# Configure TensorFlow for Auto-Growth
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5

tf.Session(config=config)
tf.set_random_seed(42) # For reproducibility

## Extracting Coffee Drinks (and other Objects) from Video

Let's perform the same task as above, but this time extract out the objects using the following video.  
* We'll also use HorovodRunner code so we can distribute this when we are processing multiple videos (or video segments).

<!--
# Can display the video from `FileStore` 
displayHTML("""<video width="600" controls>
  <source src="/dbfs/mnt/tardis6/workspace/cafe/Coffee.mp4" type="video/mp4">
</video>""")
-->

In [0]:
displayHTML("""<video width="600" controls>
  <source src="https://pages.databricks.com/rs/094-YMS-629/images/Coffee3.mp4" type="video/mp4">
</video>
""")

The following code is similar to the previous image extraction except:
* It using `detectObjectsFromVideo` instead of `detectObjectsFromImage`
* We're wrapping it with HorovodRunner so it can be distributed across multiple nodes

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard
import math
import time
import tempfile

# Use TensorFlow Backend
import tensorflow as tf
tf.set_random_seed(42) # For reproducibility

Using TensorFlow backend.

In [0]:
# Check for any available GPUs
K.tensorflow_backend._get_available_gpus()

Out[13]: 
['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [0]:
# Pin GPU to be used to process local rank (one GPU per process)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Initialize TensorFlow Session
sess = tf.Session(config=config)
K.set_session(sess)

# Re-import these libraries on the workers (vs. the driver)
from imageai.Detection import VideoObjectDetection
import os

# Configuration of execution paths
root_path = "/dbfs/ml/sbux/cafe"
root_video = "Coffee3.mp4"
execution_path = "/dbfs/ml/sbux/cafe/sources"
execution_path_export = "/dbfs/ml/sbux/cafe/"

# Bring in VideoObjectDetection
detector = VideoObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(os.path.join(root_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()

# Detect objects in video
video_path = detector.detectObjectsFromVideo(
  input_file_path=os.path.join(execution_path, root_video),
  output_file_path=os.path.join(execution_path_export, "Coffee3_detected"),
  frames_per_second=20, 
  log_progress=True
)

Processing Frame : 1
Processing Frame : 2
Processing Frame : 3
Processing Frame : 4
Processing Frame : 5
Processing Frame : 6
Processing Frame : 7
Processing Frame : 8
Processing Frame : 9
Processing Frame : 10
Processing Frame : 11
Processing Frame : 12
Processing Frame : 13
Processing Frame : 14
Processing Frame : 15
Processing Frame : 16
Processing Frame : 17
Processing Frame : 18
Processing Frame : 19
Processing Frame : 20
Processing Frame : 21
Processing Frame : 22
Processing Frame : 23
Processing Frame : 24
Processing Frame : 25
Processing Frame : 26
Processing Frame : 27
Processing Frame : 28
Processing Frame : 29
Processing Frame : 30
Processing Frame : 31
Processing Frame : 32
Processing Frame : 33
Processing Frame : 34
Processing Frame : 35
Processing Frame : 36
Processing Frame : 37
Processing Frame : 38
Processing Frame : 39
Processing Frame : 40
Processing Frame : 41
Processing Frame : 42
Processing Frame : 43
Processing Frame : 44
Processing Frame : 45
Processing Frame : 46
Processing Frame : 47
Processing Frame : 48
Processing Frame : 49
Processing Frame : 50
Processing Frame : 51
Processing Frame : 52
Processing Frame : 53
Processing Frame : 54
Processing Frame : 55
Processing Frame : 56
Processing Frame : 57
Processing Frame : 58
Processing Frame : 59
Processing Frame : 60
Processing Frame : 61
Processing Frame : 62
Processing Frame : 63
Processing Frame : 64
Processing Frame : 65
Processing Frame : 66
Processing Frame : 67
Processing Frame : 68
Processing Frame : 69
Processing Frame : 70
Processing Frame : 71
Processing Frame : 72
Processing Frame : 73
Processing Frame : 74
Processing Frame : 75
Processing Frame : 76
Processing Frame : 77
Processing Frame : 78
Processing Frame : 79
Processing Frame : 80
Processing Frame : 81
Processing Frame : 82
Processing Frame : 83
Processing Frame : 84
Processing Frame : 85
Processing Frame : 86
Processing Frame : 87
Processing Frame : 88
Processing Frame : 89
Processing Frame : 90
Processing Frame : 91
Processing Frame : 92
Processing Frame : 93
Processing Frame : 94
Processing Frame : 95
Processing Frame : 96
Processing Frame : 97
Processing Frame : 98
Processing Frame : 99
Processing Frame : 100
Processing Frame : 101
Processing Frame : 102
Processing Frame : 103
Processing Frame : 104
Processing Frame : 105
Processing Frame : 106
Processing Frame : 107
Processing Frame : 108
Processing Frame : 109
Processing Frame : 110
Processing Frame : 111
Processing Frame : 112
Processing Frame : 113
Processing Frame : 114
Processing Frame : 115
Processing Frame : 116
Processing Frame : 117
Processing Frame : 118
Processing Frame : 119
Processing Frame : 120
Processing Frame : 121
Processing Frame : 122
Processing Frame : 123
Processing Frame : 124
Processing Frame : 125
Processing Frame : 126
Processing Frame : 127
Processing Frame : 128
Processing Frame : 129
Processing Frame : 130
Processing Frame : 131
Processing Frame : 132
Processing Frame : 133
Processing Frame : 134
Processing Frame : 135
Processing Frame : 136
Processing Frame : 137
Processing Frame : 138
Processing Frame : 139
Processing Frame : 140
Processing Frame : 141
Processing Frame : 142
Processing Frame : 143
Processing Frame : 144
Processing Frame : 145
Processing Frame : 146
Processing Frame : 147
Processing Frame : 148
Processing Frame : 149
Processing Frame : 150
Processing Frame : 151
Processing Frame : 152
Processing Frame : 153
Processing Frame : 154
Processing Frame : 155
Processing Frame : 156
Processing Frame : 157
Processing Frame : 158
Processing Frame : 159
Processing Frame : 160
Processing Frame : 161
Processing Frame : 162
Processing Frame : 163
Processing Frame : 164
Processing Frame : 165
Processing Frame : 166
Processing Frame : 167
Processing Frame : 168
Processing Frame : 169
Processing Frame : 170
Processing Frame : 171
Processing Frame : 172
Processing Frame : 173
Processing Frame : 174
Processing Frame : 175
Processing Frame : 176
Processing Frame : 177
Processing Frame : 178
Processing Fra

### View Objects Detected from Video

Now that we have executed the `detectObjectsFromVideo` via ImageAI using the resnet model, let's:
* Review the output stored in the file system
* Review the objects identified in the video

<!-- 
# Can view videos directly the mount
displayHTML("""<video width="600" controls>
  <source src="/dbfs/mnt/tardis6/workspace/cafe/coffee_detected.mp4" type="video/mp4">
</video>""")
-->

In [0]:
%fs ls /ml/sbux/cafe/

path,name,size
dbfs:/ml/sbux/cafe/Coffee3_detected.avi,Coffee3_detected.avi,47507782
dbfs:/ml/sbux/cafe/resnet50_coco_best_v2.0.1.h5,resnet50_coco_best_v2.0.1.h5,152661008
dbfs:/ml/sbux/cafe/results.jpg,results.jpg,195043
dbfs:/ml/sbux/cafe/results.jpg-objects/,results.jpg-objects/,0
dbfs:/ml/sbux/cafe/sources/,sources/,0
dbfs:/ml/sbux/cafe/yolo.h5,yolo.h5,248686624


In [0]:
displayHTML("""
<video width="600" controls>
  <source src="https://pages.databricks.com/rs/094-YMS-629/images/Coffee3_detected.mp4" type="video/mp4">
</video>
""")

## Roadmap
We are starting with ImageAI due to the existing pre-trained models. Future plans include:
* Using `keras-yolov3` directly so we can train our own models to be more Starbucks specific (coffee, baked goods, cups, etc.) 
* Scale up to improve the speed of these models
* Scale out to handle more images and videos in a *distributed* fashion